In [1]:
import requests
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import time
import glob
import numpy as np
import pyodbc
import time
from sqlalchemy.sql import select
# from dask.dataframe import read_sql_query
import tqdm
import time
import json
from datetime import date, timedelta

In [2]:
today = date.today() - timedelta(days=1)
print("Today's date:", today)

Today's date: 2023-05-14


In [3]:
start = time.time()

In [4]:
url = "https://x.net/api/Login/Web"

payload = json.dumps({
  "GetMenu": True,
  "Name": "name",
  "Password": "password"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
rs_credential = json.loads(response.text)['Token']
rs_credential = 'Bearer ' + rs_credential
print('finish get new credential')

finish get new credential


# Get Invoice Data -> daily df & daily_item_df

In [6]:
today = today.strftime("%Y-%m-%d")
# today = '2023-05-14'

In [7]:
today

'2023-05-14'

In [8]:
url = "https://x.net/api/Report/ReportDetail"
## Looping for a certain date range, for now it's updated daily for today's date
# for date_temp in [x.strftime("%Y-%m-%d") for x in list(pd.date_range(start="2023-03-01",end="2023-03-27"))][:]:
print(today)
payload = json.dumps({
  "From": f"{today}T00:00:00",
  "To": f"{today}T00:00:00",
  "SalesID": "00000000-0000-0000-0000-000000000000",
  "CategoryID": "00000000-0000-0000-0000-000000000000",
  "CustomerID": "00000000-0000-0000-0000-000000000000",
  "TypeID": None,
  "ListIDs": None,
  "Bin": False,
  "CurrencyID": 1,
  "WritePerformanceLog": False,
  "Complementary": False,
  "CreatorID": "00000000-0000-0000-0000-000000000000",
  "NoteVisible": False,
  "PageNumber": 1,
  "RowCount": 100000,
  "ExportPageNumber": 1,
  "ExportRowCount": 100000,
  "SortBy": False
})
headers = {
  'Authorization': rs_credential,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

result = json.loads(response.text)
print(len(result['Data']))

list_all = []
list_item_all = []
col_header = []
col_header_item = ['id']

for i in range(0, len(result['Data'])):
    json_temp = result['Data'][i]
    list_temp = []

    for j in json_temp.keys():
        if (type(json_temp[j]) == str) or (type(json_temp[j]) == int) or (type(json_temp[j]) == float):
            list_temp.append(json_temp[j])
            if j not in col_header:
                col_header.append(j)
        else:
            pass

    list_all.append(list_temp)


    # Ini untuk informasi item
    for shop_item in json_temp['Items']:
        # Ini untuk infomasi item
        list_shop_item = []
        list_shop_item.append(json_temp['ID'])
        for k in shop_item:        
            if ((type(shop_item[k]) == str) or (type(shop_item[k]) == int) or (type(shop_item[k]) == float)) and (k!='UOMName'):
                list_shop_item.append(shop_item[k])
                if k not in col_header_item:
                    col_header_item.append(k)
        list_item_all.append(list_shop_item)

# Export aggregate data    
daily_df = pd.DataFrame(list_all, columns = col_header) 
date_info = daily_df['Date'].unique()[0][:10]
daily_df.to_csv(f'data/csv/summary/{date_info}.csv', index=False)

# Export item data
daily_item_df = pd.DataFrame(list_item_all, columns = col_header_item) 
daily_item_df.to_csv(f'data/csv/item/{date_info}.csv', index=False)


# Serializing json
json_object = json.dumps(result, indent=4)

# Writing JSON
with open(f"data/json/{date_info}.json", "w") as outfile:
    outfile.write(json_object)

# time.sleep(5)

2023-05-14
13593


# Get User -> df_id

In [9]:
url = 'https://x.net/api/Contact/CustomerList'
# for date_temp in [x.strftime("%Y-%m-%d") for x in list(pd.date_range(start="2023-03-13",end="2023-03-27"))][:]:
print(today)
payload = json.dumps({"RowCount":1000,
                      "ExportRowCount":100000,
                      "ExportPageNumber":1,
                      "From":f"{today}T00:00:00.000",
                      "To":f"{today}T00:00:00.000",
                      "ContactStateID":1,
                      "PageNumber":1})
headers = {
  'Authorization': rs_credential,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

result = json.loads(response.text)

data_temp = result['Data']
id_temp = []
id_header = ['ID', 'Name', 'Code', 'Mobile', 'Email', 'JoinDate', 'BirthDate', 'FirstName', 'LastName', 'NationalIDNumber']

for item in data_temp:
    list_temp = []
    for j in item.keys():
        list_temp.append(item[j])
    id_temp.append(list_temp)

df_id = pd.DataFrame(id_temp, columns = id_header)
df_id.to_csv(f'data/id/{today}.csv')
# time.sleep(3)

2023-05-14


# Get Report Summary -> df_rs

In [10]:
url = "https://x.net/api/Report/ReportSummary"
# for date_temp in [x.strftime("%Y-%m-%d") for x in list(pd.date_range(start="2023-3-13",end="2023-3-27"))][:]:
print(today)
payload = json.dumps({
  "CreatorID": "00000000-0000-0000-0000-000000000000",
  "CurrencyID": 0,
  "CustomerID": "00000000-0000-0000-0000-000000000000",
  "ExportPageNumber": 1,
  "ExportRowCount": 100000000,
  "ListIDs": None,
  "PageNumber": 1,
  "RowCount": 100000000,
  "From":f"{today}T00:00:00.000",
  "To":f"{today}T00:00:00.000",
  "isCreated": False
})
headers = {
  'Authorization': rs_credential,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
result = json.loads(response.text)

data_temp = result['Data']
id_temp = []
id_header = ['ID','EntityName', 'Number',  'Date','Created','CustomerName' , 'CustomerID', 'SalesName', 'Surcharge', 'Sales','Tax', 'PaymentStateID', 'SalesNTax','SurchargeAmount',  'SalesNSurcharge', 'SalesOrderNumber' ,'Cost', 'PaymentStateString','EntityID','SalesID', 'CustomerCode' ]

for item in data_temp:
    list_temp = []
    for j in item.keys():
        list_temp.append(item[j])
    id_temp.append(list_temp)

df_rs = pd.DataFrame(id_temp, columns = id_header)
df_rs.to_csv(f'data/csv/rs/{today}.csv')
# time.sleep(3)

2023-05-14


# Get Inventory

In [11]:
url = "https://x.net/api/Inventory/OutletProductList"

payload = json.dumps({
  "Date": f"{today}T10:16:39.7756148",
  "Discontinued": False,
  "QuantityFrom": 1,
  "RowCount": 10000,
  "isFilterNow": True,
  "WithOutZero": True,
  "PageNumber": 1,
  "From": "0001-01-01T00:00:00",
  "To": "0001-01-01T00:00:00"
})
headers = {
  'Authorization': rs_credential,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

result = json.loads(response.text)
data_temp = result['Data']

list_product = []
for item in data_temp:
    for outlet in item['Outlets']:
        list_outlet = [item['ID'], item['Name'], item['Code'], item['Total'], item['UOMName']]
        for detail in outlet.values():
            list_outlet.append(detail)
        list_product.append(list_outlet)
        
df_inventory = pd.DataFrame(list_product, columns = ['ID','ProductName','Code','Total','UOMName','EntityID','StoreName','Qty','In','Out','QtyNow'])
df_inventory['date'] = today
df_inventory['date'] = pd.to_datetime(df_inventory['date'])

# Insert Database

## daily item df

In [12]:
# Set up database information
server = 'server'
database = 'databasename'

conn_str = ("Driver={SQL Server};Server=server;Database=database;Trusted_Connection=yes;")

conn = pyodbc.connect(conn_str)

# conn.setdecoding(pyodbc.SQL_CHAR, encoding='latin1')
# conn.setencoding('latin1')

cursor = conn.cursor()

In [13]:
# cursor.execute('DELETE FROM dbo.rs_invoice')

In [14]:
print(daily_item_df.shape)
daily_item_df = pd.merge(daily_item_df, daily_df[['ID','PaymentStateLabel','EntityName','Date','Created','CustomerNameDisplay']], left_on='id', right_on='ID', how='left').drop('ID', axis=1)
print(daily_item_df.shape)

(29923, 10)
(29923, 15)


In [15]:
# # Check is the data already in db or not
# query = "delete from rs_agg_daily where date = '{}'".format(today)

# cursor.execute(query)
# conn.commit()

In [16]:
insert_query = "INSERT INTO dbo.rs_invoice (id,Name,Code,Quantity,UnitCost,UnitPrice,Discount,Total,Index_id,DiscountAmount,PaymentStateLabel,EntityName,Date,Created,CustomerNameDisplay) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);"
params = [(row['id'],row['Name'],row['Code'],row['Quantity'],row['UnitCost'],row['UnitPrice'], row['Discount'],row['Total'],row['Index'],row['DiscountAmount'],row['PaymentStateLabel'],row['EntityName'],row['Date'],row['Created'],row['CustomerNameDisplay']) for i, row in daily_item_df.iterrows()]
    
cursor.executemany(insert_query, params)
conn.commit()

## df_id

In [17]:
df_id['BirthDate'] = df_id['BirthDate'].str.replace('0001-01-01T00:00:00','1800-01-01')

In [18]:
if df_id.shape[0] == 0:
    print('no data')
else:
    insert_query = "INSERT INTO dbo.rs_user (ID,Name,Code,Mobile,Email,JoinDate,BirthDate,FirstName,LastName,NationalIDNumber) VALUES (?,?,?,?,?,?,?,?,?,?);"
    params = [(row['ID'],row['Name'],row['Code'],row['Mobile'],row['Email'],row['JoinDate'], row['BirthDate'],row['FirstName'],row['LastName'],row['NationalIDNumber']) for i, row in df_id.iterrows()]

    cursor.executemany(insert_query, params)
    conn.commit()

## df_rs

In [19]:
insert_query = "INSERT INTO dbo.rs_report_summary (ID,EntityName,Number_id,Date,Created,CustomerName,CustomerID,SalesName,Surcharge,Sales, Tax, PaymentStateID, SalesNTax, SurchargeAmount, SalesNSurcharge, SalesOrderNumber, Cost, PaymentStateString, EntityID, SalesID, CustomerCode) VALUES (?,?,?,?,?,?,?,?,?,?, ?,?,?,?,?,?,?,?,?,?, ?);"
params = [(row['ID'],row['EntityName'],row['Number'],row['Date'],row['Created'],row['CustomerName'], row['CustomerID'],row['SalesName'],row['Surcharge'],row['Sales'],row['Tax'],row['PaymentStateID'],row['SalesNTax'],row['SurchargeAmount'],row['SalesNSurcharge'],row['SalesOrderNumber'],row['Cost'],row['PaymentStateString'],row['EntityID'],row['SalesID'],row['CustomerCode']) for i, row in df_rs.iterrows()]

cursor.executemany(insert_query, params)
conn.commit()

## df_inventory

In [20]:
insert_query = "INSERT INTO dbo.rs_inventory (id,product_name,code,total,uom,entity_id,name,qty,qty_in,qty_out, qty_now, date) VALUES (?,?,?,?,?,?,?,?,?,?,?,?);"
params = [(row['ID'],row['ProductName'],row['Code'],row['Total'],row['UOMName'],row['EntityID'], row['StoreName'],row['Qty'],row['In'],row['Out'],row['QtyNow'],row['date']) for i, row in df_inventory.iterrows()]

cursor.executemany(insert_query, params)
conn.commit()

## df_agg

In [21]:
daily_item_df['Total'] = daily_item_df['Quantity']*daily_item_df['UnitPrice']

In [22]:
df_agg = daily_item_df.groupby(['Date','EntityName','Code'])[['Quantity','Total']].sum().reset_index()

In [23]:
df_agg.head()
#example of agg data 

,Date,EntityName,Code,Quantity,Total
0,2023-05-14T00:00:00,ANTAPANI,0505160002,1,10000
1,2023-05-14T00:00:00,ANTAPANI,300875127674,1,347000
2,2023-05-14T00:00:00,ANTAPANI,4008976682417,2,530000
3,2023-05-14T00:00:00,ANTAPANI,4391852988117,1,14500
4,2023-05-14T00:00:00,ANTAPANI,4801010561309,2,11000


## Insert to DB 

In [ ]:
insert_query = "INSERT INTO dbo.rs_agg_daily (Date, EntityName, Code, Quantity, Total) VALUES (?,?,?,?,?);"
params = [(row['Date'],row['EntityName'],row['Code'],row['Quantity'],row['Total']) for i, row in df_agg.iterrows()]

cursor.executemany(insert_query, params)
conn.commit()

In [25]:
end = time.time()

In [26]:
print(end - start)

255.18994116783142


In [27]:
(end - start)/60

4.253165686130524

In [28]:
----stop----

NameError: name 'stop' is not defined